# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
mars_soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
rows = mars_soup.find_all('tr')

# Optional: Print the rows to see the structure
for row in rows:
    print(row)

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
data_rows = []
# Loop through the scraped data to create a list of rows
for row in rows[1:]:  # Skip the first row as it contains headers
   # Extract data from each row
    cells = row.find_all('td')
    # Create a dictionary to store the row data
    row_data = {
        'id': cells[0].text,
        'terrestrial_date': cells[1].text,
        'sol': cells[2].text,
        'ls': cells[3].text,
        'month': cells[4].text,
        'min_temp': cells[5].text,
        'pressure': cells[6].text
    }
    
    # Append the row data to the list of rows
    data_rows.append(row_data)
    print(data_rows)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
# Create a list of column names
column_names = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']

# Create a Pandas DataFrame
mars_df = pd.DataFrame(data_rows, columns=column_names)


In [ ]:
# Confirm DataFrame was created successfully
# Display the DataFrame
print(mars_df)

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
print(mars_df.dtypes)

In [ ]:
# Change data types for data analysis
# Convert 'terrestrial_date' column to datetime
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'])

# Convert other columns to appropriate data types
mars_df['id'] = mars_df['id'].astype('int64')
mars_df['sol'] = mars_df['sol'].astype('int64')
mars_df['ls'] = mars_df['ls'].astype('int64')
mars_df['month'] = mars_df['month'].astype('int64')
mars_df['min_temp'] = mars_df['min_temp'].astype('float64')
mars_df['pressure'] = mars_df['pressure'].astype('float64')

In [ ]:
# Confirm type changes were successful by examining data types again
# Print the updated data types
print(mars_df.dtypes)

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
# Count the occurrences of each unique month
month_counts = mars_df['month'].value_counts().sort_index()

# Print the output
print(month_counts)


In [ ]:
# 2. How many Martian days' worth of data are there?
# Count the number of unique sol values
num_sols = mars_df['sol'].nunique()

# Print the output
print("Number of Martian days' worth of data:", num_sols)

In [ ]:
# 3. What is the average low temperature by month?
# Calculate the average low temperature by month
average_low_temp_by_month = mars_df.groupby('month')['min_temp'].mean()

# Print the output
print(average_low_temp_by_month)


In [ ]:
# Plot the average temperature by month in bar chart format
average_low_temp_by_month.plot(kind='bar', figsize=(10, 6), color='skyblue')

# Set the title and labels
plt.title('Average Low Temperature by Month on Mars')
plt.xlabel('Month')
plt.ylabel('Average Low Temperature (°C)')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Identify the coldest and hottest months in Curiosity's location
# Find the coldest and hottest months
coldest_month = average_low_temp_by_month.idxmin()
hottest_month = average_low_temp_by_month.idxmax()

# Plot the average temperature by month in bar chart format
ax = average_low_temp_by_month.plot(kind='bar', figsize=(10, 6), color='skyblue')

# Highlight the coldest and hottest months
ax.axvline(x=coldest_month - 1, color='blue', linestyle='--', label='Coldest Month')
ax.axvline(x=hottest_month - 1, color='red', linestyle='--', label='Hottest Month')

# Set the title and labels
plt.title('Average Low Temperature by Month on Mars')
plt.xlabel('Month')
plt.ylabel('Average Low Temperature (°C)')

# Add a legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# 4. Average pressure by Martian month
# Calculate the average pressure by Martian month
average_pressure_by_month = mars_df.groupby('month')['pressure'].mean()

# Print the output
print(average_pressure_by_month)


In [ ]:
# Plot the average pressure by month
# Plot the average pressure by month in bar chart format
average_pressure_by_month.plot(kind='bar', figsize=(10, 6), color='lightgreen')

# Set the title and labels
plt.title('Average Pressure by Martian Month')
plt.xlabel('Month')
plt.ylabel('Average Pressure (Pa)')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Plot daily minimum temperature against the number of terrestrial days
plt.figure(figsize=(10, 6))

# Plot the daily minimum temperature against the index (number of terrestrial days)
plt.plot(mars_df.index, mars_df['min_temp'], color='blue', label='Daily Minimum Temperature')

# Set the title and labels
plt.title('Daily Minimum Temperature vs. Number of Terrestrial Days')
plt.xlabel('Number of Terrestrial Days')
plt.ylabel('Daily Minimum Temperature (°C)')

# Add grid lines
plt.grid(True)

# Show legend
plt.legend()

# Show plot
plt.tight_layout()
plt.show()


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
# Define the file path
csv_file_path = "mars_data.csv"

# Write the DataFrame to a CSV file
mars_df.to_csv(csv_file_path, index=False)

print("Data has been successfully written to", csv_file_path)


In [ ]:
browser.quit()